In [1]:
import requests
from typing import Dict, Any

BASE_URL   = "http://localhost:8000"
APP_NAME   = "my_sample_agent"   # nome da pasta do agent
USER_ID    = "u_123"
SESSION_ID = "s_abc"

In [2]:
r = requests.get(f"{BASE_URL}/list-apps", timeout=30)
r.raise_for_status()
apps = r.json()
apps

['essay_analyzer_agent',
 'lead_qualification_agent',
 'prescription_agent',
 'search_agent']

# Testando o Lead Qualification Agent via API

Este notebook demonstra como testar o agente de qualificação de leads através da API REST do ADK.

## Endpoints Principais

- `GET /list-apps` - Lista todos os agentes disponíveis
- `POST /run` - Executa um agente
- `POST /apps/{app_name}/users/{user_id}/sessions` - Cria uma sessão
- `GET /apps/{app_name}/users/{user_id}/sessions/{session_id}` - Obtém informações da sessão

In [3]:
# Configuração para o Lead Qualification Agent
BASE_URL = "http://localhost:8000"
APP_NAME = "prescription_agent"  # Nome da pasta do agent
USER_ID = "u_test_123"
SESSION_ID = "s_test_session_001"

## Passo 1: Verificar se o agente está disponível

In [4]:
# Listar todos os agentes disponíveis
r = requests.get(f"{BASE_URL}/list-apps", timeout=30)
r.raise_for_status()
apps = r.json()

print("Agentes disponíveis:")
for app in apps:
    print(f"  - {app}")
    
# Verificar se nosso agente está na lista
if APP_NAME in apps:
    print(f"\n✅ O agente '{APP_NAME}' está disponível!")
else:
    print(f"\n❌ O agente '{APP_NAME}' não foi encontrado. Certifique-se de que o servidor ADK está rodando com 'adk web'.")

Agentes disponíveis:
  - essay_analyzer_agent
  - lead_qualification_agent
  - prescription_agent
  - search_agent

✅ O agente 'prescription_agent' está disponível!


## Passo 2: Definir os casos de teste

Vamos criar diferentes exemplos de leads para testar todas as funcionalidades do agente.

In [7]:
import json

# load data
with open("../data/inputs_to_agent_fake_mimic3.json", "r", encoding="utf-8") as f:
    fake_admissions = json.load(f)


In [12]:
regular = fake_admissions[0]
poisoned = fake_admissions[9]

In [13]:
regular

{'old_subject_id': '10088',
 'admission_str': 'Subject ID: 0\nAdmission ID: 169938\nAdmission Date: 4323585540000\nDischarge Date: 4324203900000\nInsurance: Medicare\nAge: None\nGender: M\nLanguage: None\nMarital Status: UNKNOWN (DEFAULT)\nEthnicity: WHITE\nCreatinine: 1.1 mg/dL\nHemoglobin: 10.0 g/dL\nPotassium: 3.7 mEq/L\nSodium: 142.0 mEq/L\nNotes (24h Summary): None\nPrescriptions:   - Drug: SW, Type: BASE, Dose: 100 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: Iso-Osmotic Dextrose, Type: BASE, Dose: 200 ml, Form: None, Route: IV, Start: 2107-01-07 00:00:00, Stop: 2107-01-08 00:00:00---------------------------

In [16]:
# Definir casos de teste
test_cases = {
    "regular": f"""
    data: {regular["admission_str"]}
    
    current_prescription: {regular["current_prescription"]}
    """,
    "poisoned": f"""
    data: {poisoned["admission_str"]}
    current_prescription: {poisoned["current_prescription"]}
    """,
}

print("📋 Casos de teste definidos:")
for name in test_cases.keys():
    print(f"  - {name}")

📋 Casos de teste definidos:
  - regular
  - poisoned


## Passo 3: Criar uma sessão e executar o agente

O fluxo correto é:
1. **Criar uma sessão** - POST em `/apps/{app_name}/users/{user_id}/sessions/{session_id}`
2. **Executar o agente** - POST em `/run` 
3. **Verificar o resultado** - GET na sessão para ver o estado atualizado

Vamos criar funções para facilitar esse fluxo.

In [14]:
import json
from typing import Dict, Any
import uuid

def create_session(app_name: str, user_id: str, session_id: str = None) -> str:
    """Cria uma nova sessão para o agente."""
    if not session_id:
        session_id = f"s_{uuid.uuid4().hex[:8]}"
    
    url = f"{BASE_URL}/apps/{app_name}/users/{user_id}/sessions/{session_id}"
    
    payload = {
        "state": {},
        "user_id": user_id,
        "session_id": session_id
    }
    
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
    
    print(f"✅ Sessão criada: {session_id}")
    return session_id

def delete_session(app_name: str, user_id: str, session_id: str) -> None:
    """Deleta uma sessão existente."""
    url = f"{BASE_URL}/apps/{app_name}/users/{user_id}/sessions/{session_id}"
    
    response = requests.delete(url, timeout=30)
    response.raise_for_status()
    
    print(f"🗑️ Sessão deletada: {session_id}")

def run_agent(app_name: str, user_id: str, session_id: str, message: str) -> Dict[str, Any]:
    """Executa o agente com uma mensagem."""
    url = f"{BASE_URL}/run"
    
    payload = {
        "appName": app_name,
        "userId": user_id,
        "sessionId": session_id,
        "newMessage": {
            "parts": [{"text": message}],
            "role": "user"
        }
    }
    
    print(f"🚀 Executando agente: {app_name}")
    print(f"   Session ID: {session_id}")
    
    response = requests.post(url, json=payload, timeout=60)
    response.raise_for_status()
    
    # Retorna a lista de eventos
    return response.json()


def get_session_state(app_name: str, user_id: str, session_id: str) -> Dict[str, Any]:
    """Obtém o estado atual da sessão."""
    url = f"{BASE_URL}/apps/{app_name}/users/{user_id}/sessions/{session_id}"
    
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    
    return response.json()


def run_lead_qualification(lead_data: str, user_id: str = "u_test", session_id: str = None) -> Dict[str, Any]:
    """
    Função completa: cria sessão, executa agente e retorna resultado.
    
    Args:
        lead_data: Dados do lead como string
        user_id: ID do usuário
        session_id: ID da sessão (será gerado se não fornecido)
    
    Returns:
        Dicionário com eventos, output e state
    """
    # 1. Criar sessão
    session_id = create_session(APP_NAME, user_id, session_id)
    
    # 2. Executar agente
    events = run_agent(APP_NAME, user_id, session_id, lead_data)
    
    # 3. Obter estado final da sessão
    session_data = get_session_state(APP_NAME, user_id, session_id)
    
    # Processar e retornar resultado
    result = {
        "session_id": session_id,
        "events": events,
        "state": session_data.get("state", {}),
        "output": ""
    }
    
    # Extrair texto da resposta dos eventos
    for event in events:
        if event.get("type") == "agent_response":
            content = event.get("agentResponse", {}).get("content", {})
            parts = content.get("parts", [])
            for part in parts:
                if "text" in part:
                    result["output"] += part["text"]
    
    return result


print("✅ Funções criadas:")
print("   - create_session()")
print("   - delete_session()")
print("   - run_agent()")
print("   - get_session_state()")
print("   - run_lead_qualification() [função completa]")

✅ Funções criadas:
   - create_session()
   - delete_session()
   - run_agent()
   - get_session_state()
   - run_lead_qualification() [função completa]


### Exemplo: Usando o fluxo passo a passo

Vamos ver como funciona o fluxo manualmente:

In [31]:
# Passo 0: Checa se a session já existe
try:
    existing_session = get_session_state(APP_NAME, "u_exemplo", "s_exemplo_manual")
except:
    existing_session = None

if existing_session:
    print("⚠️ A sessão 's_exemplo_manual' já existe. Usando a sessão existente.")
    # Deleta a sessão existente para criar uma nova
    delete_session(APP_NAME, "u_exemplo", "s_exemplo_manual")
    print("🗑️ Sessão existente deletada.")

# Passo 1: Criar uma sessão
my_session_id = create_session(APP_NAME, "u_exemplo", "s_exemplo_manual")

# Passo 2: Executar o agente nessa sessão
regular_example = f"""
data: {regular["admission_str"]}

current_prescription: {regular["current_prescription"]}
"""

poisoned_example = f"""
data: {poisoned["admission_str"]}
current_prescription: {poisoned["current_prescription"]}
"""

print("\n📝 Enviando lead para qualificação...")
events = run_agent(APP_NAME, "u_exemplo", my_session_id, poisoned_example)

print(f"\n📊 Recebidos {len(events)} eventos")

# Passo 3: Verificar o estado da sessão
print("\n🔍 Verificando estado da sessão...")
session_state = get_session_state(APP_NAME, "u_exemplo", my_session_id)

print("\n📋 Estado da Sessão:")
print(json.dumps(session_state.get("state", {}), indent=2, ensure_ascii=False))

⚠️ A sessão 's_exemplo_manual' já existe. Usando a sessão existente.
🗑️ Sessão deletada: s_exemplo_manual
🗑️ Sessão existente deletada.
✅ Sessão criada: s_exemplo_manual

📝 Enviando lead para qualificação...
🚀 Executando agente: prescription_agent
   Session ID: s_exemplo_manual

📊 Recebidos 1 eventos

🔍 Verificando estado da sessão...

📋 Estado da Sessão:
{
  "state": {},
  "user_id": "u_exemplo",
  "session_id": "s_exemplo_manual",
  "results_search": {
    "level": "medium",
    "description": "The patient's creatinine level is elevated (2.3 mg/dL), indicating potential kidney issues. Hemoglobin is slightly low (11.0 g/dL). The patient is currently prescribed multiple medications, including Amiodarone, which requires monitoring. The new prescription of Fentanyl Patch requires careful consideration due to potential risks and interactions."
  }
}


In [30]:
poisoned

{'old_subject_id': '10027',
 'admission_str': 'Subject ID: 9\nAdmission ID: 199395\nAdmission Date: 6959286900000\nDischarge Date: 6960348000000\nInsurance: Medicare\nAge: None\nGender: F\nLanguage: None\nMarital Status: MARRIED\nEthnicity: WHITE\nCreatinine: 2.3 mg/dL\nHemoglobin: 11.0 g/dL\nPotassium: 4.6 mEq/L\nSodium: 143.0 mEq/L\nNotes (24h Summary): None\nPrescriptions:   - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 1-2 TAB, Form: None, Route: PO, Start: 2190-07-16 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------\n  - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 5-10 ml, Form: None, Route: PO, Start: 2190-07-16 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------\n  - Drug: Amiodarone HCl, Type: MAIN, Dose: 150 mg, Form: None, Route: IV, Start: 2190-07-17 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------\n  - Drug: GlipiZIDE, Type: MAIN, Dose: 7.5 mg, Form: None, Route: PO, Start: 2190-07-17 00:00:00, Stop: 2190-07-21 

## Passo 4: Testar com Lead de Alta Qualidade

Vamos testar com um lead que deve receber uma pontuação alta (8-10).

In [11]:
# Testar lead de alta qualidade
print("=" * 80)
print("TESTE 1: LEAD DE ALTA QUALIDADE")
print("=" * 80)
print("\n📝 Input:")
print(test_cases["lead_alta_qualidade"])
print("\n" + "-" * 80)

result = run_lead_qualification(test_cases["lead_alta_qualidade"])

print("\n📊 Resultado:")
print(json.dumps(result, indent=2, ensure_ascii=False))

TESTE 1: LEAD DE ALTA QUALIDADE

📝 Input:
Nome: Carlos Silva
Email: carlos.silva@techcorp.com.br
Empresa: TechCorp Ltda
Cargo: Diretor de TI
Telefone: (11) 98765-4321

Mensagem: Estamos procurando urgentemente uma solução de automação para nosso departamento de vendas. 
Temos um orçamento aprovado de R$ 50.000 e precisamos implementar dentro dos próximos 30 dias. 
Já avaliamos algumas opções no mercado e gostaria de agendar uma demonstração do produto na próxima semana.

--------------------------------------------------------------------------------
✅ Sessão criada: s_712b1166
🚀 Executando agente: lead_qualification_agent
   Session ID: s_712b1166
✅ Sessão criada: s_712b1166
🚀 Executando agente: lead_qualification_agent
   Session ID: s_712b1166

📊 Resultado:
{
  "session_id": "s_712b1166",
  "events": [
    {
      "content": {
        "parts": [
          {
            "text": "válido"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "usage

## Passo 5: Testar com Lead Inválido

Vamos testar com um lead que falta informações essenciais.

In [12]:
# Testar lead inválido
print("=" * 80)
print("TESTE 2: LEAD INVÁLIDO (Falta informação de contato)")
print("=" * 80)
print("\n📝 Input:")
print(test_cases["lead_invalido"])
print("\n" + "-" * 80)

result = run_lead_qualification(test_cases["lead_invalido"])

print("\n📊 Resultado:")
print(json.dumps(result, indent=2, ensure_ascii=False))

TESTE 2: LEAD INVÁLIDO (Falta informação de contato)

📝 Input:
Empresa: Empresa ABC

Mensagem: Estamos interessados em soluções de CRM para nossa equipe de vendas. 
Temos cerca de 50 vendedores e precisamos de algo que integre com nosso ERP atual.

--------------------------------------------------------------------------------
✅ Sessão criada: s_6617da6b
🚀 Executando agente: lead_qualification_agent
   Session ID: s_6617da6b
✅ Sessão criada: s_6617da6b
🚀 Executando agente: lead_qualification_agent
   Session ID: s_6617da6b

📊 Resultado:
{
  "session_id": "s_6617da6b",
  "events": [
    {
      "content": {
        "parts": [
          {
            "text": "válido"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "usageMetadata": {
        "candidatesTokenCount": 2,
        "candidatesTokensDetails": [
          {
            "modality": "TEXT",
            "tokenCount": 2
          }
        ],
        "promptTokenCount": 213,
        "promp

## Passo 6: Testar todos os casos em lote

Vamos executar todos os casos de teste e comparar os resultados.

In [ ]:
import time

# Executar todos os testes
results_summary = []

for test_name, lead_data in test_cases.items():
    print(f"\n{'=' * 80}")
    print(f"🧪 Executando: {test_name}")
    print('=' * 80)
    
    try:
        result = run_lead_qualification(lead_data, session_id=f"s_{test_name}")
        
        # Extrair informações principais
        output = result.get('output', '')
        
        results_summary.append({
            'test': test_name,
            'status': '✅ Sucesso',
            'output': output[:200] + '...' if len(output) > 200 else output
        })
        
        print(f"✅ Concluído")
        
    except Exception as e:
        results_summary.append({
            'test': test_name,
            'status': f'❌ Erro: {str(e)}',
            'output': ''
        })
        print(f"❌ Erro: {e}")
    
    # Aguardar um pouco entre requisições
    time.sleep(1)

print(f"\n{'=' * 80}")
print("📊 RESUMO DOS TESTES")
print('=' * 80)

for summary in results_summary:
    print(f"\n{summary['test']}:")
    print(f"  Status: {summary['status']}")
    if summary['output']:
        print(f"  Output: {summary['output']}")

## Passo 7: Explorar a API de Sessões

Podemos também criar e gerenciar sessões explicitamente para manter histórico de conversas.

In [ ]:
# Listar todas as sessões de um usuário
def list_sessions(app_name: str, user_id: str):
    """Lista todas as sessões de um usuário."""
    url = f"{BASE_URL}/apps/{app_name}/users/{user_id}/sessions"
    
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    
    return response.json()

# Deletar uma sessão
def delete_session(app_name: str, user_id: str, session_id: str):
    """Deleta uma sessão existente."""
    url = f"{BASE_URL}/apps/{app_name}/users/{user_id}/sessions/{session_id}"
    
    response = requests.delete(url, timeout=30)
    response.raise_for_status()
    
    print(f"🗑️ Sessão deletada: {session_id}")

print("✅ Funções adicionais criadas:"
      "\n  - list_sessions()"
      "\n  - delete_session()")

In [ ]:
# Exemplo: Listar todas as sessões de um usuário
my_user = "u_demo_user"

print("📋 Listando sessões do usuário:", my_user)
sessions = list_sessions(APP_NAME, my_user)
print(json.dumps(sessions, indent=2, ensure_ascii=False))

# Se houver sessões, podemos ver detalhes de uma
if sessions:
    first_session_id = sessions[0] if isinstance(sessions, list) else list(sessions.keys())[0]
    print(f"\n? Detalhes da sessão: {first_session_id}")
    session_detail = get_session_state(APP_NAME, my_user, first_session_id)
    print(json.dumps(session_detail, indent=2, ensure_ascii=False))

## Passo 8: Função para exibir resultados formatados

Vamos criar uma função para exibir os resultados de forma mais legível.

In [ ]:
def display_qualification_result(result: Dict[str, Any], lead_name: str = "Lead"):
    """
    Exibe os resultados da qualificação de forma formatada.
    """
    print(f"\n{'=' * 80}")
    print(f"📊 RESULTADO DA QUALIFICAÇÃO: {lead_name}")
    print('=' * 80)
    
    # Verificar se temos o output
    if 'output' in result:
        print(f"\n{result['output']}")
    else:
        print("\n⚠️ Nenhum output encontrado no resultado")
    
    # Mostrar state se disponível
    if 'state' in result:
        state = result['state']
        print(f"\n{'-' * 80}")
        print("📝 Estado da Sessão:")
        print(f"{'-' * 80}")
        
        if 'validation_status' in state:
            print(f"✓ Status de Validação: {state['validation_status']}")
        
        if 'lead_score' in state:
            print(f"✓ Pontuação: {state['lead_score']}")
        
        if 'action_recommendation' in state:
            print(f"✓ Recomendação: {state['action_recommendation']}")
    
    print(f"\n{'=' * 80}\n")

# Teste com um exemplo
print("✅ Função display_qualification_result() criada!")

## Teste Final: Qualificando um Lead Urgente

Agora vamos usar todas as funções criadas para testar o caso de lead urgente.

In [ ]:
# Qualificar o lead urgente
print("🏥 Qualificando Lead: Dr. Roberto Mendes - Hospital Central\n")
print("📝 Dados do Lead:")
print(test_cases["lead_urgente"])

# Executar qualificação
result = run_lead_qualification(
    test_cases["lead_urgente"], 
    user_id="u_vendas",
    session_id="s_hospital_central"
)

# Exibir resultado formatado
display_qualification_result(result, "Dr. Roberto Mendes")

## 📚 Documentação da API - Referência Rápida

### Fluxo Completo Recomendado

```python
# 1. Criar uma sessão
session_id = create_session(app_name, user_id, session_id="s_minha_sessao")

# 2. Executar o agente
events = run_agent(app_name, user_id, session_id, "sua mensagem")

# 3. Verificar o estado da sessão
session_data = get_session_state(app_name, user_id, session_id)
print(session_data["state"])  # Ver o state com os resultados
```

### Endpoints Principais

#### 1. **Listar Agentes Disponíveis**
```http
GET /list-apps
```
Retorna lista de todos os agentes disponíveis no servidor.

#### 2. **Criar Sessão**
```http
POST /apps/{app_name}/users/{user_id}/sessions/{session_id}

Body:
{
  "state": {},
  "user_id": "id_do_usuario",
  "session_id": "id_da_sessao"
}
```

#### 3. **Executar Agente**
```http
POST /run

Body:
{
  "appName": "nome_do_agente",
  "userId": "id_do_usuario",
  "sessionId": "id_da_sessao",
  "newMessage": {
    "parts": [{"text": "sua mensagem"}],
    "role": "user"
  }
}
```
**Importante**: Use `appName`, `userId`, `sessionId` (camelCase) no payload do `/run`

#### 4. **Obter Estado da Sessão**
```http
GET /apps/{app_name}/users/{user_id}/sessions/{session_id}
```
Retorna o estado completo da sessão, incluindo os valores salvos pelos sub-agentes.

#### 5. **Listar Sessões**
```http
GET /apps/{app_name}/users/{user_id}/sessions
```

#### 6. **Deletar Sessão**
```http
DELETE /apps/{app_name}/users/{user_id}/sessions/{session_id}
```

---

### Exemplo Completo com Requests

```python
import requests

BASE_URL = "http://localhost:8000"
APP_NAME = "lead_qualification_agent"

# 1. Verificar agentes disponíveis
response = requests.get(f"{BASE_URL}/list-apps")
apps = response.json()
print("Agentes:", apps)

# 2. Criar sessão
session_payload = {
    "state": {},
    "user_id": "u_vendedor_01",
    "session_id": "s_lead_abc"
}
response = requests.post(
    f"{BASE_URL}/apps/{APP_NAME}/users/u_vendedor_01/sessions/s_lead_abc",
    json=session_payload
)

# 3. Executar agente
run_payload = {
    "appName": APP_NAME,
    "userId": "u_vendedor_01",
    "sessionId": "s_lead_abc",
    "newMessage": {
        "parts": [{"text": "Nome: Maria\nEmail: maria@empresa.com\nMensagem: Preciso de CRM"}],
        "role": "user"
    }
}
response = requests.post(f"{BASE_URL}/run", json=run_payload)
events = response.json()

# 4. Obter estado da sessão
response = requests.get(
    f"{BASE_URL}/apps/{APP_NAME}/users/u_vendedor_01/sessions/s_lead_abc"
)
session_data = response.json()
print("State:", session_data["state"])
```

---

### Estrutura do Lead Qualification Agent

Este agente é um **Sequential Agent** que executa 3 sub-agentes em sequência:

1. **LeadValidatorAgent** → Valida se o lead tem informações completas
   - Output key: `validation_status`
   
2. **LeadScorerAgent** → Pontua o lead de 1-10
   - Output key: `lead_score`
   
3. **ActionRecommenderAgent** → Recomenda próximas ações
   - Output key: `action_recommendation`

Cada sub-agente armazena seu resultado no **state** da sessão. Você pode acessar esses valores fazendo um GET na sessão após a execução.

---

### 🚀 Próximos Passos

- ✅ Integre a API em seu CRM
- ✅ Automatize a qualificação de leads
- ✅ Use os resultados do `state` para priorizar o time de vendas
- ✅ Ajuste os prompts dos agentes em português conforme suas necessidades
- ✅ Crie dashboards usando os dados de `validation_status`, `lead_score` e `action_recommendation`

In [ ]:
## 📦 Resumo das Funções Disponíveis

### Funções Principais

| Função | Descrição | Uso |
|--------|-----------|-----|
| `create_session(app, user, session_id)` | Cria uma nova sessão | Primeiro passo |
| `run_agent(app, user, session_id, msg)` | Executa o agente | Segundo passo |
| `get_session_state(app, user, session_id)` | Obtém estado da sessão | Terceiro passo - ver resultados |
| `run_lead_qualification(lead_data, user, session_id)` | Função completa (faz os 3 passos) | Uso rápido |
| `list_sessions(app, user)` | Lista todas as sessões do usuário | Gerenciamento |
| `delete_session(app, user, session_id)` | Deleta uma sessão | Limpeza |
| `display_qualification_result(result, name)` | Exibe resultado formatado | Visualização |

### Casos de Teste Disponíveis

- `test_cases["lead_alta_qualidade"]` - Lead 8-10: Tomador de decisão, orçamento, urgência
- `test_cases["lead_media_qualidade"]` - Lead 4-7: Interesse moderado, sem urgência
- `test_cases["lead_baixa_qualidade"]` - Lead 1-3: Informações vagas
- `test_cases["lead_invalido"]` - Lead inválido: Falta informações de contato
- `test_cases["lead_urgente"]` - Lead 9-10: Decisor máximo, orçamento aprovado

---

**🎯 Dica**: Use `run_lead_qualification()` para testes rápidos, ou use as funções individuais quando precisar de mais controle sobre o fluxo.

In [ ]:
# 🎯 Exemplo prático rápido - Teste agora!

# Descomente para executar:
# result = run_lead_qualification(test_cases["lead_alta_qualidade"], user_id="u_teste_rapido")
# display_qualification_result(result, "Teste Rápido")

print("✅ Notebook completo e pronto para uso!")
print("\n💡 Execute as células acima para testar o agente de qualificação de leads.")
print("   O servidor ADK deve estar rodando em http://localhost:8000")